In [1]:
from hello import hello, hello3
from time import sleep

In [4]:
def call_v3(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except hello3.NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv
        
def call_v2(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    fail = 0
    while True:
        try:
            rv = f(*args)
        except Exception:
            if fail == 1:
                raise
            h.login()
            fail = 1
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv
        
def mg(sp):
    return call(h.setmg, 1, sp)

def off():
    call(h.setmg, 2, 0)
    call(h.setdo, 2, 0, 0)
    call(h.setph, 2, 0, 0)

def getconfig():
    global co2max, airmin, n2min, o2min, co2min
    cfg = call(h.getConfig)
    co2max = cfg['pH']['CO2_Manual_Max_(%)']
    
    airmin = cfg['Gas_Data']['Air_Min_(LPM)']
    n2min = cfg['Gas_Data']['N2_Min_(LPM)']
    o2min = cfg['Gas_Data']['O2_Min_(LPM)']
    co2min = cfg['Gas_Data']['CO2_Min_(LPM)']
    
    call(h.setconfig, 'pH', 'CO2_Manual_Max_(%)', 100)
    
def fixconfig():
    call(h.setconfig, 'pH', 'CO2_Manual_Max_(%)', co2max)
    
def n2on():
    call(h.setdo, 1, 100, 0)
    mg(n2min)
    
def o2on_v2():
    call(h.setdo, 1, 0, o2min*1000)
    
def o2on_v3():
    call(h.setdo, 1, 0, 100)
    mg(o2min)
    
def airon():
    mg(airmin)
    
def co2on():
    call(h.setph, 1, 100, 0)
    mg(co2min)
    
def verify(v):
    assert input("Verify Flow (%s) "%v) != "n"
    
def main(ip, v):
    global call, o2on
    global h
    
    if v == 2:
        call = call_v2
        K = hello.HelloApp
        o2on = o2on_v2
    else:
        call = call_v3
        K = hello3.HelloApp
        o2on = o2on_v3
        
    h = K(ip)
    getconfig()
    for f in airon, n2on, co2on, o2on:
        off()
        sleep(4)
        f()
        verify(f.__name__[:-2])
    fixconfig()

In [5]:
main('192.168.1.12', 3)

Calling getConfig()
Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
Calling setmg(2, 0)
Calling setdo(2, 0, 0)
Calling setph(2, 0, 0)
Calling setmg(1, 0.03)
Verify Flow (air) 
Calling setmg(2, 0)
Calling setdo(2, 0, 0)
Calling setph(2, 0, 0)
Calling setdo(1, 100, 0)
Calling setmg(1, 0.03)
Verify Flow (n2) 
Calling setmg(2, 0)
Calling setdo(2, 0, 0)
Calling setph(2, 0, 0)
Calling setph(1, 100, 0)
Calling setmg(1, 0.03)
Verify Flow (co2) 
Calling setmg(2, 0)
Calling setdo(2, 0, 0)
Calling setph(2, 0, 0)
Calling setdo(1, 0, 100)
Calling setmg(1, 0.02)
Verify Flow (o2on_) 
Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100.0)
